# Homework 

*Please view this notebook with Appmode turned off.*

### Instructions 

In this exercise, you'll perform a series of MD simulations involving charged particles in the presence of an electromagnetic field. The main simulation code is provided for you. For each computational question, you will:
* Copy and paste relevant code into a new code cell,
* Make modifications (as directed),
* Run the simulation, and 
* Report on your observations in Markdown-formatted text.

There is also a single non-coding question at the end of the homework. Answer this question in a separate Markdown cell. 

### Expected content

Answers to questions 1 - 3 should include:
* Working simulation code in a cell block immediately below the question statement.
* All output generated by your simulation code (don't "clear output" before you save your notebook).
* A Markdown-formatted answer to the question immediately below the simulation code output. Blank Markdown cells are provided: double click in the text "Type Markdown and LaTeX: $\alpha^2$" just below the relevant simulation code block. 

The answer to question 4 should consist of a Markdown code block only. 

### Provided code

There are two main code blocks needed for this exercise: 
* **MD Setup**: The code block *immediately below this cell* and labeled "Simulation Setup" needs to be executed only once. It imports relevant libraries and defines useful functions and constants needed for the MD simulation. You do **not** need to copy this code in the answers to your questions since it will never be modified. 

* **MD Run**: The code block provided *immediately below Question 1* runs the MD simulation. All parameters are initially set to the values needed to answer Question 1. You will then copy and paste this code into the cell blocks below Questions 2 and 3, modify it as needed, and use the modified code to answer the questions. 


### Simulation Setup: Execute this code only once.

In [3]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time

import numpy as np

def calc_accel(x,y,efield):
    ax = 4*epso*(Ro**4)/(M*x**5) - 4*epso*(Ro**4)/(M*(L-x)**5)
    ay = 4*epso*(Ro**4)/(M*y**5) - 4*epso*(Ro**4)/(M*(L-y)**5)
    for p1 in range(0, Npart):
        for p2 in range(0,Npart):
            if p1!=p2:
                rX = x[p1] - x[p2]
                rY = y[p1] - y[p2]
                r = math.sqrt(rX*rX + rY*rY)
                ax[p1] += Q[p1]*Q[p2]*rX/(M*r**3)
                ay[p1] += Q[p1]*Q[p2]*rY/(M*r**3)
                ax[p1] += 12.0*epso*rX*(Ro**12)/(M*r**14) - 12.0*epso*rX*(Ro**6)/(M*r**8)
                ay[p1] += 12.0*epso*rY*(Ro**12)/(M*r**14) - 12.0*epso*rY*(Ro**6)/(M*r**8)
    ay += Q*efield/M
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay,efield):
    xnew = x + vx*dt + 0.5*ax*dt*dt
    ynew = y + vy*dt + 0.5*ay*dt*dt
    axnew,aynew = calc_accel(xnew,ynew,efield)
    vxnew = vx + 0.5*(ax + axnew)*dt
    vynew = vy + 0.5*(ay + aynew)*dt
    return xnew,ynew,vxnew,vynew,axnew,aynew

def init_plot():
    fig = plt.figure(1)
    ax1 = plt.clf()
    txt = plt.text(0.1*L,0.1*L,'t = '+str(round(0))+' fs')
    negLine, = plt.plot(X[0:Npos],Y[0:Npos],'bo')
    posLine, = plt.plot(X[Npos:],Y[Npos:],'ro')
    plt.xlim([0,L])
    plt.ylim([0,L])
    plt.xticks([]) 
    plt.yticks([]) 
    
    ax2 = plt.axes([1.0,0.575,0.3,0.3])
    plt.xlabel('$t$')
    plt.ylabel('$E(t)$')
    field_line, = plt.plot(taxis,Efield)
    plt.xticks([])
    plt.yticks([])
    plt.ylim([0,1])
    plt.ylim([0,dt*Nsteps])
    return fig,ax1,ax2,negLine,posLine,txt,field_line

def update_plot(n):
    negPts.set_ydata(Y[0:Npos])
    negPts.set_xdata(X[0:Npos])
    posPts.set_ydata(Y[Npos:])
    posPts.set_xdata(X[Npos:])
    txt.set_text('t = '+str(round(n*dt*1e+15))+' fs')
    
    plt.sca(ax2)
    #plt.plot(taxis[0:n]/Emax, 'k.')
    field_line.set_xdata(taxis[0:n])
    field_line.set_ydata(Efield[0:n]/Emax)
    plt.ylim([-1,1])
    #plt.xlim([0,dt*Nsteps])
    
    fig.canvas.draw()
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
def gauss_pulse(t):
    return Emax*np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)
Ro = 5.0e-8

## Question 1

Using the code below, run a simulation with 5 positive particles and 5 negative particles. Initially the particle positions are random; positive and negative particle positions are independent of each other. Does the pulse tend to create correlations between the particle positions? (E.g., do positive and negative particles move in the same or different directions?) Why? Does the correlation last after the pulse is gone? (E.g., in the final frame, do the particle positions look random?) 


In [ ]:
# SIMULATION PARAMETERS #
Npos = 5
Nneg = 5
L = 2e-6
to = 500e-15
sigma = 100e-15
nu = 2e+12
Emax = 100e+4   # Maximum electric field in statV/cm

tmax=1.5e-12      # Total simulation time in seconds
dt=0.25e-15     # Time-step in seconds
Nsteps=int(round(tmax/dt))
M=4*(1.66054e-24)   # Mass in g
Qo = 4.803e-10         # Elementary charge in statCoulombs
taxis = np.arange(0,tmax,dt)   # Time axis (array of time steps)


# Set Particle charges
Npart = Npos+Nneg
Q = np.zeros((Npart))  # Empty vector for particle charges
Q[0:Npos] = +Qo    # First Npos particles are positive
Q[Npos:] = -Qo     # Last Nneg particles are negative

# Generate a vector of random particle positions
X = 0.1*L + np.random.random((Npart))*0.8*L
Y = 0.1*L + np.random.random((Npart))*0.8*L

# Set initial velocities to zero 
VX = np.zeros((Npart))
VY = np.zeros((Npart))

# Generate the pulse profile
Efield = gauss_pulse(taxis)

# Calculate accelerations at initial positions
AX,AY = calc_accel(X,Y,Efield[0])

fig,ax1,ax2,negPts,posPts,txt,field_line = init_plot()
for n in range(0,Nsteps):
    X,Y,VX,VY,AX,AY = vv_step(X,Y,VX,VY,AX,AY,Efield[n])
    if(n%50==0):
        update_plot(n)
        

## Question 2

Run a simulation with:
* A single positive particle having initial position (x,y) = (0.5*L, 0.5*L) and zero velocity (i.e., you should only have **one particle** in your simulation box)
* The box length (L) set to 1 nm (note that simulation units are cm)
* A pulse frequency $\nu$ of 10 THz (note: program frequency units are in Hz = 1/s)

Finally, inside the ``if(n%50==0):`` block, add the line ``plt.plot(n*dt,Y[0]/L-0.5,'r.')`` to plot the change in $y$-coordinate of the particle (normalized to the box length) alongside the electric field. Run the simulation again. 

Is the change in position of the particle in sync or out of sync with the field? Why?

In [32]:
# Insert code for Question 2 here. 
# Insert Markdown answers in the box below. 

## Question 3

Repeat the single-particle simulation with a pulse frequency of $\nu = 5$ THz. What is different about the response of the particle when $\nu = 10$ THz (Question 2) vs $\nu = 5$ THz (Question 3)? Which one leaves the particle in a "more perturbed" state at the end of the simulation? Why? 



## Question 4

In our simulation, the variable ``Efield`` represents an "external field" applied to the sample from, e.g., a focused laser beam. In reality, of course, the electric field in the box is much more complicated, since each charged particle produces its own local field. Why did we not include this in calculating the Lorentz force on each particle? Under what conditions is this treatment appropriate? 




# Extra Credit #

Consider a Gaussian pulse propagating along the $z$ axis with an electric field
\begin{align}
{\boldsymbol E}({\boldsymbol x}, t) ={\boldsymbol E}^{(\text{max})} e^{- \frac{x^2+y^2}{2 \sigma_x^2}} e^{- \frac{\left (t - \frac{n z}{c} \right)^2 }{ 2 \sigma_t^2} } \cos \omega \left( \frac{nz}{c} - t \right ) .
\end{align}
If $\left \| {\boldsymbol E}^{(\text{max})} \right \| = 1 \cdot 10^4 {\text{statV/cm}}$, $\omega = 2\pi c \cdot 1650$ cm$^{-1}$, $\sigma_t = 20 $ fs, and $\sigma_x = 25 \mu$m, what is the pulse energy? Express the answer in both cgs and SI units. 